In [1]:
!pip install pyomo
!pip install haversine


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.7 MB/s eta 0:00:00


In [2]:
#Install Solvers
%%capture
import sys
import os

if True:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [3]:
import numpy as np
import pandas as pd
import os
import pyomo.environ as pyo
from haversine import haversine, Unit
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#airport data (already cleaned)
#change to desired path
ap = pd.read_csv(os.path.join("/","content", "drive", "MyDrive", "OPTI_Final", "DataC", "arAirports.csv"))
ap["co"] = ap.apply(lambda row: (row.lat, row.lon),axis=1)
ap.index = ap.index + 1
#county data (already cleaned)
#change to desired path
co = pd.read_csv(os.path.join("/","content", "drive", "MyDrive", "OPTI_Final", "DataC", "arCounties.csv"))
co["co"] = co.apply(lambda row: (row.lat, row.lon),axis=1)
co.index = co.index + 1

In [5]:
#Sets
I = co.index.to_list()
J = ap.index.to_list()

#Params
D = {}
for i in I:
    county_loc = co.at[i, "co"]
    for j in J:
        airport_loc = ap.at[j, "co"]
        D[(i,j)] = haversine(county_loc, airport_loc, unit=Unit.MILES)

A = {i:co.at[i, "pop"] for i in I}

In [6]:
model = pyo.ConcreteModel()

In [7]:
#county index
model.I = pyo.Set(initialize=I)
#airport index
model.J = pyo.Set(initialize=J)

In [8]:
print(I[0], I[-1])
print(J[0], J[-1])

1 75
1 7


In [9]:
#distance between airports & counties
model.d = pyo.Param(model.I, model.J, initialize=D)
#population supply of the counties
model.a = pyo.Param(model.I, initialize=A)
#number of facilities to fly out of
model.p = pyo.Param(mutable=True, initialize=1)

In [10]:
model.X = pyo.Var(model.I, model.J, domain=pyo.Binary)
model.Y = pyo.Var(model.J, domain=pyo.Binary)

In [11]:
def obj_rule(model):
    return(sum(sum(model.a[i]*model.d[(i,j)]*model.X[(i,j)] for j in model.J) for i in model.I))

model.obj = pyo.Objective(rule=obj_rule(model), sense=pyo.minimize)

In [12]:
model.facility_demand_con = pyo.ConstraintList()
for i in model.I:
    model.facility_demand_con.add(sum(model.X[i, j] for j in model.J) == 1)

In [13]:
def num_facilities_constraint(model):
    return(sum(model.Y[j] for j in model.J) == model.p)
model.num_facilities_con = pyo.Constraint(rule=num_facilities_constraint(model))

In [14]:
model.demand_supplied_con = pyo.ConstraintList()
for j in model.J:
    for i in model.I:
        model.demand_supplied_con.add(model.X[(i,j)] <= model.Y[j])

In [15]:
solver = pyo.SolverFactory('cbc')
solver.solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 256218129.66539758
  Upper bound: 256218129.66539758
  Number of objectives: 1
  Number of constraints: 601
  Number of variables: 532
  Number of binary variables: 532
  Number of integer variables: 532
  Number of nonzeros: 525
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.06
  Wallclock time: 0.06
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  St

In [16]:
model.Y.pprint()

Y : Size=7, Index=J
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary
      3 :     0 :   0.0 :     1 : False : False : Binary
      4 :     0 :   0.0 :     1 : False : False : Binary
      5 :     0 :   0.0 :     1 : False : False : Binary
      6 :     0 :   0.0 :     1 : False : False : Binary
      7 :     0 :   1.0 :     1 : False : False : Binary


In [17]:
for j in model.J:
  if pyo.value(model.Y[j]) == 1:
    print(f"Good airport: {j}, {ap.at[j, 'Fac_Name']}")

Good airport: 7, BILL AND HILLARY CLINTON NATIONAL/ADAMS FIELD
